# wczytywanie

In [1]:
from parse import parse_task_1_2

In [2]:
sent, sentences = parse_task_1_2('training/file_39.ttl', only_sentences=True)
print(sent)

Knowledge Services and Dotstaff is incredibly happy to be a part of this wonderful community, said Joe Bielawski, the president of Knowledge Services.


# parsowanie

In [4]:
from entities_recognition_tester import process_using_spacy
import en_core_web_sm

In [5]:
pary = process_using_spacy(sent, en_core = en_core_web_sm.load(), format_result=True)
print(pary)

[('Dotstaff', 'ORG'), ('Joe Bielawski', 'PERSON'), ('Knowledge Services', 'ORG')]


In [47]:
knowladge = [
    # coś, coś, relacja, słowa kluczowe do rozpoznania relacji
    ['ORG', 'ORG', 'affiliation', ['relate']],
    ['Person', 'EducationalInstitution',  'almaMater', ['study']],
    ['Band', 'Person', 'bandMember', ['play', 'music']],
    ['Person', 'Place', 'birth', ['birth', 'born']],
    ['Person', 'ORG', 'ceo', ['president', 'boss', 'head', 'chief']],
    ['Person', 'Person', 'child', ['child']],
    ['Athlete', 'SportsTeam', 'club', ['train', 'play']],
    ['ORG', 'COUNTY', 'country', ['locate']],
    ['Person', 'COUNTY', 'country', ['from']],
    ['Place', 'COUNTY', 'country', ['in']],
    ['Person', 'Place',  'death', ['death']],
    ['ATHLETE', 'SPOERTSTEAM', 'debutTeam', ['first']],
    ['PopulatedPlace', 'PopulatedPlace', 'department', ['from', 'in']],
    ['Place', 'PopulatedPlace', 'district', ['in']],
    ['Scientist', 'Person', 'doctoralAdvisor', ['medical']],
    ['Scientist', 'Person', 'doctoralStudent', ['doctoral', 'doctoral']],
    ['Person', 'ORG', 'employer', ['work', 'worker']],
    ['Band', 'Person', 'formerBandMember', []],
    ['Athlete', 'SportsTeam', 'formerTeam', ['played']],
    ['ORG', 'City', 'foundationPlace', ['in', 'locate']],
    ['ORG', 'PopulatedPlace', 'headquarter', ['in']],
    ['ORG', 'Settlement', 'hometown', ['from']],
    ['Person', 'Settlement', 'hometown', ['from', 'live']],  
    ['PopulatedPlace', 'Person', 'leaderName', []],
    ['Place', 'Place', 'locatedInArea', ['from', 'in']],
    ['Person', 'Place', 'location', ['at']],
    ['ORG', 'Place', 'location', ['at']],
    ['Place', 'Place', 'location', ['at']],
    ['Person', 'Country', 'nationality', ['from']],  
    ['Person', 'Person', 'parent', ['child']],  
    ['ORG', 'Person', 'president', ['president']],
    ['Person', 'ORG', 'president', ['president']],
    ['Person', 'Person', 'relative', ['relate']],
    ['Person', 'Person', 'spouse', ['wife', 'husband']], 
    ['Company', 'Company', 'subsidiary', []],
    ['ArchitecturalStructure', 'ORG', 'tenant', ['rent']],
    ['Athlete', 'Person', 'trainer', ['train']]
]



In [54]:
import copy
zdanie = copy.copy(sent)

for item in pary:
    zdanie = zdanie.replace(item[0], item[1])

def in_first(word):
    to_return = []
    for item in knowladge:
        if word.capitalize() == item[0].capitalize():
            to_return.append(item)
    return to_return

zdanie = zdanie.replace(',', ' ')
zdanie = zdanie.replace('.', ' ')
zdanie = zdanie.replace(';', ' ')
zdanie = zdanie.split(' ')
for i in range(0, len(zdanie)):
    tmp = in_first(zdanie[i])
    if tmp != []:
        string = zdanie[i+1::]
        for item in tmp:
            if item[1] in string:
                pos = string.index(item[1])
                string = string[0:pos]
                #print(string)
                for kw in item[3]:
                    if kw in string:
                        print(item[2]) #nazwa relacji
                        print(zdanie[i:])
            

ceo
['PERSON', '', 'the', 'president', 'of', 'ORG', '']


In [32]:
from sparql_resource_property_getter import get_resource

[print(get_resource(think)) for think, label in pary]

[]
[]
[]


[None, None, None]

# relacja

In [5]:
from word_word_relation import get_relation

In [25]:
rel = get_relation(pary, sent)
[print(r) for r in rel]

(('Dotstaff', 'ORG'), ('Joe Bielawski', 'PERSON'), 'president')
(('Knowledge Services', 'ORG'), ('Joe Bielawski', 'PERSON'), 'president')


[None, None]

# odpytywania

In [7]:
from sparql_resource_property_getter import get_info

In [8]:
in_db = get_info(rel[1])
print(in_db)

[False, False]


# zwracanie

In [9]:
from formatuj import formatuj

In [10]:
formatuj(rel = rel[1], 
         in_db = in_db, 
         sent = sent, 
         meta = sentences[sent])

<https://www.indystar.com/story/news/local/hamilton-county/2017/06/06/indianapolis-company-moving-headquarters-fishers-adding-400-jobs/373808001/#char=0,18> ;
a  nif:RFC5147String , nif:String , nif:Phrase ;
nif:anchorOf  "Knowledge Services"^^xsd:string ;
nif:beginIndex  "0"^^xsd:nonNegativeInteger ;
nif:endIndex  "18"^^xsd:nonNegativeInteger ;
nif:referenceContext <https://www.indystar.com/story/news/local/hamilton-county/2017/06/06/indianapolis-company-moving-headquarters-fishers-adding-400-jobs/373808001/#char=0,150> ;
itsrdf:taIdentRef http://aksw.org/notInWiki/Knowledge_Services .

<https://www.indystar.com/story/news/local/hamilton-county/2017/06/06/indianapolis-company-moving-headquarters-fishers-adding-400-jobs/373808001/#char=99,112> ;
a  nif:RFC5147String , nif:String , nif:Phrase ;
nif:anchorOf  "Joe Bielawski"^^xsd:string ;
nif:beginIndex  "99"^^xsd:nonNegativeInteger ;
nif:endIndex  "112"^^xsd:nonNegativeInteger ;
nif:referenceContext <https://www.indystar.com/story/news/